 - Input the data
 - Create a field for today (10th August 2022)
 - Create a data field to show how much capital is paid off each month
 - Create a data field to show how many months are needed to pay off the entire debt (whole months only)
 - Create a field when the mortgages will be paid off by (Assuming a payment is to be made in August 2022)
 - Create a row per month between now and when the mortgage is paid off showing:
 - How much is still to be paid off for that mortgage? Call this field 'Remaining Capital to Repay'
 - How much is still to be paid off for all mortgages? Call this field ' Capital Outstanding Total'
 - Rename the date field 'Monthly Payment Date'
 - Output the data

In [1]:
import os
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd

In [2]:
def clean_dframe(df):
     df.columns = [i.strip().lower().replace(' ','_') for i in df.columns]
     return df
td = dt.date(2022,8,10) # Use dates in example

In [3]:
df = clean_dframe(pd.read_csv('preppin_data_20220813.csv'))
df['today'] = td
df['monthly_capital_payment'] = df['monthly_payment'] * (df['%_of_monthly_repayment_going_to_capital']/100) # How much capital do you pay each month?
df['months_to_pay'] = (df['capital_repayment_remaining']/ df['monthly_capital_payment']) # How many monts do you still have to pay?
df['months_to_pay']=df['months_to_pay'].astype('int') #Change from float to int
df['total_capital_repayment_remaining'] = df['capital_repayment_remaining'].sum() #How much do you owe for both properties?
df['months_to_pay']=df['months_to_pay'].apply(lambda x:pd.DateOffset(months=x)) #change dtype to a time delta. Will use this to calculate end date.
df['end_date']=df['today']+df['months_to_pay'] # Canlulate end date


# Create date table

In [4]:
# Get dates
range_start = df['today'].min()
range_end = df['end_date'].max().date() + relativedelta(months=1)

# Create dframe
all_dates_df = pd.DataFrame(pd.date_range(start=range_start,end=range_end,freq='m'),columns=['dates'])
# The above function returns month end dates. In the below I calculate back to the original dates
date_diff = range_start - all_dates_df.min()[0].date() #Get time delta
all_dates_df['dates']=all_dates_df['dates'] + date_diff #Change date
all_dates_df['dates']=all_dates_df['dates'].dt.date #Convert from timestamt to date

df.drop(columns=['monthly_payment','%_of_monthly_repayment_going_to_capital','months_to_pay','today'],inplace=True)#remove excess columns

# Combine dates table with data table
Uses cross join

In [5]:
# Add fields for cross join
df['j'] = 1
all_dates_df['j'] = 1
combine_df = df.merge(all_dates_df,how='outer',on='j').sort_values('dates').drop(columns=['j']).reset_index(drop=True).groupby(['store','dates','end_date']).sum()

# Filter out dates where loans are fully paid
combine_df.reset_index(inplace=True) 
combine_df=combine_df[combine_df['dates']<=combine_df['end_date']].copy()
combine_df.set_index(['store','dates','end_date'],inplace=True)#Set index back so that the group by works below.

In [6]:
combine_df['capital_paid'] = combine_df.groupby(['store'])['monthly_capital_payment'].cumsum() # Running sum of paid debt per store
total_cap_cumsum = combine_df.groupby(['dates'])['monthly_capital_payment'].sum().cumsum() #Running sum of paid debt for total(First sum, then cumsum - order of operations)
combine_df=combine_df.reset_index().merge(total_cap_cumsum,how='left',on='dates',suffixes=['','_total']).set_index(['store','dates','end_date'])# Merge total cumsum to DF
combine_df['capital_outstanding_total'] = combine_df['total_capital_repayment_remaining']-combine_df['monthly_capital_payment_total'] #Calculate total debt remaining by month
combine_df['remaining_capital_to_repay'] = combine_df['capital_repayment_remaining']-combine_df['capital_paid']#Calculate debt remaining for each store by month

In [7]:
#Output data
combine_df.droplevel(level=2)[[ 'capital_outstanding_total', 'remaining_capital_to_repay']].to_csv('output.csv')
os.startfile('output.csv')